In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error

Nous cherchons ici a élaboré un model de ML qui va nous permettre de savoir si un pays se situe dans la tranche des pays
les plus heureux ou au contraire dans celle des pays les moins heureux.

In [2]:
#import des 2 csv

df = pd.read_csv('world-happiness-report-2021.csv')

df2 = pd.read_csv('world-happiness-report.csv')
df2.head()

#Création liste des régions du monde par pays
country_region = df.iloc[:,0:2]

#Mise en forme des 2 datasets en prévision du concat
#dataset de 2021 :
df_1 = df.iloc[:,:12]
df_1 = df_1.drop(df.iloc[:,3:6], axis=1)
df_1 = df_1.drop('Regional indicator', axis=1)
df_1['year']='2021'

#dataset historique depuis 2005 :
df_2 = df2.iloc[:, :-2]
dico = {'Life Ladder' : 'Ladder score', 'Log GDP per capita' : 'Logged GDP per capita', 'Healthy life expectancy at birth' : 'Healthy life expectancy'}
df_2.rename(dico, axis=1, inplace=True)

#création du fichier contenants les données depuis 2005 :
df_full = pd.concat([df_1, df_2], axis=0)
df_full['year'] = df_full['year'].astype('int')
df_full = df_full.sort_values(by='Country name', ascending=True)

#ajout colonne Region d'après le df country_region créé précédemment :
df_full = df_full.merge(country_region, on = 'Country name', how ='inner')

#ajout du rank par année de chaque pays
df_full["rank"] = df_full.groupby("year")["Ladder score"].rank("dense", ascending=False).astype('int')

df_full.head()
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2035 entries, 0 to 2034
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Country name                  2035 non-null   object 
 1   Ladder score                  2035 non-null   float64
 2   Logged GDP per capita         2011 non-null   float64
 3   Social support                2026 non-null   float64
 4   Healthy life expectancy       1984 non-null   float64
 5   Freedom to make life choices  2005 non-null   float64
 6   Generosity                    1959 non-null   float64
 7   Perceptions of corruption     1931 non-null   float64
 8   year                          2035 non-null   int32  
 9   Regional indicator            2035 non-null   object 
 10  rank                          2035 non-null   int32  
dtypes: float64(7), int32(2), object(2)
memory usage: 174.9+ KB


In [3]:

#Remplacement des NaN par les moyennes des variables en fonction de chaque pays
for i in df_full.iloc[:,2:8].columns : 
    df_full[i] = df_full[i].fillna(df_full.groupby('Country name')[i].transform("mean"))

In [4]:
#transformation de la variable Ladder Score en 2 catéogire. Catégorie 1, pays les plus heureux, 2: pays les moins heureux.

def filtre(x):
    if x >= 0 and x <= 5.5 :
        return '2'
    if x > 5.5 and x <= 10 :
        return '1'

df_full['Ladder score_categ'] = df_full['Ladder score'].apply(filtre)

In [5]:
# variable cile

target = df_full['Ladder score_categ']

#création d'un DataFrame contenant uniquement les features

col_feat = ['Logged GDP per capita', 'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption']



rlm_df_full = df_full[col_feat]

In [6]:
#Standardisation des données


scaler = StandardScaler()
data = scaler.fit_transform(rlm_df_full)



In [7]:
# Séparation des données en deux jeux train et test:

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=666)


Un découpage des features par composante via ACP a été tenté, 2 composantes ressortaient mais cela a réduit l'efficacité du model de quelques %, cette approche a donc été laissé de côté.



Pour le choix du model, 3 variantes ont été testés: SVM, RandomForest et KNN. Le SVM a été éliminé rapidement car très long 
sur la recherche des best paramétres (plusieurs heures), Le randomforest ayant donné les meilleurs résultats,
nous l avons donc conservé comme model final.

In [8]:
#Création d'un model de RandomForest + GridSearch afin de trouver les meilleurs hyperparamètres 

clf = RandomForestClassifier(random_state=666)
l = np.arange(100,400,10)
param = {"max_features" : ["sqrt", "log2"], "n_estimators": l, "criterion" :["gini", "entropy"]}

grid = GridSearchCV(estimator=clf,param_grid=param)
grid.fit(X_train,y_train)
print(grid.best_params_)


{'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 310}


In [9]:
#création d'un model de RanDomForest basé sur les best paramètres:

clf_best = RandomForestClassifier(criterion="entropy", max_features="sqrt", n_estimators = 310, random_state =666)

clf_best.fit(X_train,y_train)
y_pred = clf_best.predict(X_test)

print('score test :', clf_best.score(X_test, y_test))
print('mse test :', mean_squared_error(y_pred, y_test))

score test : 0.9017199017199017
mse test : 0.09828009828009827


In [10]:
pd.crosstab(y_test, y_pred, rownames = ['Classe réelle'], colnames = ['Classe prédite'])

Classe prédite,1,2
Classe réelle,,
1,159,16
2,24,208


In [11]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.87      0.91      0.89       175
           2       0.93      0.90      0.91       232

    accuracy                           0.90       407
   macro avg       0.90      0.90      0.90       407
weighted avg       0.90      0.90      0.90       407



Une accurracy global de 90%, model très performant 